In [ ]:
import os
import pandas as pd
import openai
import numpy as np
import faiss
from tqdm import tqdm
import pandas as pd
import pickle
import gradio as gr


# Подключение к OpenAI API
openai.api_key = os.getenv("OPENAI_API_KEY", "")

print("Импорты успешно выполнены.")


Импорты успешно выполнены.


In [49]:
parts = []

process = lambda text: text.strip().lower()

for i in range(3):

    part = pd.read_csv(f"Harry Potter {i+1}.csv", sep=';')
    part.columns = [*map(str.lower, part.columns)]

    part.iloc[:, 0] = part.iloc[:, 0].apply(process)
    part.iloc[:, 1] = part.iloc[:, 1].apply(process)

    parts.append(part)

all_parts_df = pd.concat(parts, axis=0)
all_parts_df.drop_duplicates(inplace=True)
all_parts_df.reset_index(drop=True, inplace=True)
all_parts_df.tail(20)

,character,sentence
4743,seamus,"after you, of course."
4744,ron,quiet.
4745,ron,let the man through.
4746,ron,"i didn't mean to open it, harry."
4747,ron,it was badly wrapped.
4748,ron,they made me do it.
4749,fred & george,did not.
4750,george,it's a firebolt.
4751,fred,it's the fastest broom in the world.
4752,harry,for me?


**Возьмем реплики Гарри Поттера**

In [4]:
harry_replicas = all_parts_df[all_parts_df.character == "harry"].sentence.astype(str).tolist()
harry_replicas[:10]

['yes, aunt petunia.',
 'yes, uncle vernon.',
 "he's asleep!",
 'sorry about him.',
 "he doesn't understand what it's like, lying there day after day...",
 '...watching people press their ugly faces in on you.',
 'can you hear me?',
 "it's just, i've never talked to a snake before.",
 'do you...?',
 'i mean, do you talk to people often?']

**Векторизацию будем проводить с помощью OpenAI**

In [5]:
import os


texts_sample = harry_replicas

embeddings_file = "all_embeddings.npy"

if os.path.exists(embeddings_file):
    all_embeddings = np.load(embeddings_file)
    print("Эмбеддинги загружены из файла.")
else:
    client = openai.OpenAI(api_key=openai.api_key)  # Новый способ вызова API

    all_embeddings = []
    for text in tqdm(texts_sample, desc="Embedding texts"):
        response = client.embeddings.create(
            input=text,
            model="text-embedding-ada-002"  # здесь размер эмбеддинга по дефолту 1536, это может быть избыточно для коротких фраз
        )
        emb = response.data[0].embedding
        all_embeddings.append(emb)

    all_embeddings = np.array(all_embeddings, dtype='float32')
    np.save(embeddings_file, all_embeddings)
    print("Эмбеддинги вычислены и сохранены в файл.")

print("Размер массива эмбеддингов:", all_embeddings.shape)


Эмбеддинги загружены из файла.
Размер массива эмбеддингов: (1028, 1536)


**Создание индекса Faiss**

##### Мы хотим искать близкие реплики по косинусному сходству. Faiss использует либо евклидово расстояние, либо скалярное произведение (dot product). Для косинусной близости можно сначала нормализовать вектора:

In [6]:
# Нормализуем
norms = np.linalg.norm(all_embeddings, axis=1, keepdims=True)
emb_normed = all_embeddings / norms

dim = emb_normed.shape[1]

# Создаём индекс для поиска по dot-product (что эквивалентно косинусному сходству после нормализации)
index = faiss.IndexFlatIP(dim)

# Добавляем эмбеддинги в индекс
index.add(emb_normed)

print(f"Всего векторов в индексе: {index.ntotal}")


Всего векторов в индексе: 1028


**Функция для «поиска ответа»**

In [7]:
def get_closest_reply(user_text, top_k=1):
    # 1) Эмбеддинг вопроса
    client = openai.OpenAI(api_key=openai.api_key)
    resp = client.embeddings.create(input=user_text, model="text-embedding-ada-002")
    query_emb = np.array(resp.data[0].embedding, dtype='float32')
    
    # 2) Нормализуем
    query_emb_normed = query_emb / np.linalg.norm(query_emb)
    
    # 3) Поиск в Faiss
    D, I = index.search(query_emb_normed.reshape(1, -1), top_k)
    # D - массив сходств, I - массив индексов
    
    # 4) Получим тексты
    # texts_sample - это первые MAX_EMBED реплик, которые мы реально индексировали.
    # Нужно брать по индексам из I.
    
    results = []
    for rank in range(top_k):
        idx = I[0][rank]
        sim = D[0][rank]
        text = texts_sample[idx]
        results.append((text, sim))
    
    return results

# Тестируем
test_query = "How are you?"
result = get_closest_reply(test_query, top_k=5)
for r in result:
    print("REPLY:", r[0], "| SIMILARITY=", r[1])


REPLY: and you? | SIMILARITY= 0.85968596
REPLY: who are you? | SIMILARITY= 0.8581377
REPLY: you okay? | SIMILARITY= 0.8550978
REPLY: hello? | SIMILARITY= 0.85434425
REPLY: hello? | SIMILARITY= 0.85434425


**Последние 2 ответа повторяются. В предобработке данных необходимо также проверить на дубли**



**Коллега на занятии отметил, что правильно будет искать не близкую запросу реплику, а реплику на которую отвечает персонаж**

##### Также преобразуем наш код в более правильную форму, так как почти все выше описанне действия включает в себя один компонент - Retriever

In [ ]:
class OpenAIRetriever:
    def __init__(self, model_name: str = "text-embedding-ada-002"):
        """
        Инициализация ретривера с использованием OpenAI Embeddings.
        
        Параметры:
        - model_name: имя модели OpenAI для получения эмбеддингов (по умолчанию "text-embedding-ada-002").
        
        Создается клиент OpenAI через конструкцию:
            client = openai.OpenAI(api_key=openai.api_key)
        """
        self.model_name = model_name
        # Инициализируем клиента OpenAI
        self.client = openai.OpenAI(api_key=openai.api_key)
        self.contexts = []   # Список контекстов (реплики, на которые отвечает Гарри)
        self.responses = []  # Список ответов Гарри (реплики, сказанные Гарри)
        self.index = None    # Faiss индекс для контекстов
        self.dim = None      # Размерность эмбеддингов

    def __getstate__(self):
        """
        При сериализации объекта исключает атрибут client.
        """
        state = self.__dict__.copy()
        del state['client']
        return state

    def __setstate__(self, state):
        """
        При десериализации объекта инициализирует атрибут client.
        """
        self.__dict__.update(state)
        self.client = openai.OpenAI(api_key=openai.api_key)

    def load_dialogue_data(self, df: pd.DataFrame) -> None:
        """
        Обрабатывает DataFrame для формирования пар "контекст–ответ".
        Если текущая реплика принадлежит Гарри, а предыдущая — нет, то:
          - предыдущая реплика становится контекстом,
          - текущая реплика становится ответом.
          
        После формирования пар вызывается load_corpus для индексирования контекстов. 
        """
        self.contexts.clear()
        self.responses.clear()
        
        # Итерируем по DataFrame, начиная со второй строки
        for i in range(1, len(df)):
            prev_row = df.iloc[i - 1]
            curr_row = df.iloc[i]
            # Приводим имена персонажей к нижнему регистру и убираем лишние пробелы
            if curr_row['character'].strip().lower() == 'harry' and prev_row['character'].strip().lower() != 'harry':
                self.contexts.append(prev_row['sentence'])
                self.responses.append(curr_row['sentence'])
        print(f"Всего выбрано контекстов: {len(self.contexts)}")
                
        # После формирования пар индексируем контексты
        self.load_corpus(self.contexts)

    def load_corpus(self, contexts: list) -> None:
        """
        Вычисляет эмбеддинги для списка контекстов с использованием OpenAI API и строит Faiss индекс.
        Перед поиском эмбеддинги нормализуются, чтобы inner product работал как косинусное сходство.
        """
        # Получаем эмбеддинги для контекстов
        embeddings = self._get_embeddings(contexts)
        # Нормализуем векторы (каждый вектор делим на его L2-норму)
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        embeddings_norm = embeddings / norms
        
        self.dim = embeddings_norm.shape[1]
        # Создаем Faiss индекс с inner product (при нормализации это косинусное сходство)
        self.index = faiss.IndexFlatIP(self.dim)
        self.index.add(embeddings_norm.astype('float32'))

    def _get_embeddings(self, texts: list) -> np.ndarray:
        """
        Получает эмбеддинги для списка текстов через OpenAI API.
        
        texts: список строк.
        
        Возвращает numpy-массив эмбеддингов (dtype float32).
        """
        response = self.client.embeddings.create(
            input=texts,
            model=self.model_name
        )
        # Извлекаем эмбеддинги для каждого текста
        embeddings = [item.embedding for item in response.data]

        print(f"Число запрошенных текстов: {len(texts)}")
        print(f"Число возвращённых эмбеддингов: {len(response.data)}")
        return np.array(embeddings, dtype='float32')
    
    def encode_query(self, query: str) -> np.ndarray:
        """
        Получает эмбеддинг для запроса пользователя и нормализует его.
        
        query: строка запроса.
        
        Возвращает эмбеддинг запроса в виде массива размерности (1, dim).
        """
        response = self.client.embeddings.create(
            input=[query],
            model=self.model_name
        )
        embedding = np.array(response.data[0].embedding, dtype='float32')
        normalized = embedding / np.linalg.norm(embedding)
        return normalized.reshape(1, -1)
    
    def search(self, query: str, top_k: int = 5) -> list:
        """
        Выполняет поиск по запросу.
        
        1. Получает эмбеддинг запроса.
        2. Ищет top_k наиболее похожих контекстов в Faiss индексе.
        3. Возвращает список ответов (реплик Гарри), соответствующих найденным контекстам.
        
        query: строка запроса от пользователя.
        top_k: количество результатов для возврата.
        """
        query_embedding = self.encode_query(query)
        distances, indices = self.index.search(query_embedding, top_k)
        results = []
        for idx in indices[0]:
            if idx < len(self.responses):
                results.append(self.responses[idx])
        return results

    def get_answer(self, query: str) -> str:
        """
        Возвращает лучший ответ (первый найденный) для заданного запроса.
        Если результаты отсутствуют, возвращает сообщение-заглушку.
        """
        results = self.search(query, top_k=1)
        return results[0] if results else "Извините, я не нашёл подходящего ответа."

# Пример использования:
# Допустим, у вас есть DataFrame all_parts_df с колонками 'character' и 'sentence'
# (например, сформированный в результате объединения CSV файлов).
retriever = OpenAIRetriever()
retriever.load_dialogue_data(all_parts_df)
# Сохранение объекта retriever в файл
with open('retriever.pkl', 'wb') as file:
    pickle.dump(retriever, file)

Всего выбрано контекстов: 600
Число запрошенных текстов: 600
Число возвращённых эмбеддингов: 600


In [60]:
query = "Harry, how are you?"
answer = retriever.get_answer(query)
print("Ответ:", answer)

Ответ: hey, hagrid.


In [ ]:
with open('retriever.pkl', 'rb') as file:
    retriever = pickle.load(file)

# Функция, которая отвечает на пользовательское сообщение, вызывая  retriever
def chat_fn(user_message):
    # Получаем ответ от ретривера
    bot_answer = retriever.get_answer(user_message)  
    return bot_answer

#  Создание Gradio интерфейса
chatbot = gr.ChatInterface(
    fn=chat_fn,            # функция, которая будет обрабатывать новые сообщения
    title="Гарри Поттер Бот",
    description="Задай вопрос боту и получи ответ от Гарри Поттера!",
    # Можно добавить несколько примеров (примеров ввода) для кнопок "Examples":
    examples=["Hi, Harry!", "What do you think about magic?"],
)

# Запуск локального сервера Gradio
chatbot.launch(server_name="0.0.0.0")  # Let Gradio choose a free port



c:\Users\betsu\Miniconda3\Lib\site-packages\gradio\chat_interface.py:334: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7860): обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7861): обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7862): обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт)
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 7863): обычно разрешается только одно использование адреса сокета (протокол/сетевой адрес/порт

* Running on local URL:  http://0.0.0.0:7865

To create a public link, set `share=True` in `launch()`.


**В общем получилось плохо. Долго мучился, но так и не понял, почему не получилось реализовать ответ. Возможно неудаяный датасет и выборка маленькая, всего 600 пар, где реплика Гарри следует за чьей-то другой**